# Portfolio Optimization 

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/qci-wdyk/eqc-models-tutorial/blob/main/tutorial07-portfolio-optimization.ipynb)


Select a value for $\xi$ and solve the program
$$
\begin{equation}
    \min_{\{w_{i}\}_{i \in \{1, 2,..., K\}}} [-E(R) R_{B} + \xi \mathrm{VAR}(R)]
\end{equation}
$$

to select a weighted proportion of the portfolio to seek maximum return and minimized risk.


In [1]:
import sys
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from eqc_models.allocation import PortMomentum
from eqc_models.solvers import Dirac3ContinuousCloudSolver
from utils import (
    get_nasdaq100_constituents,
    get_port_stats,
)

In [2]:
# Set parameters
ADJ_DATE = "2022-01-01"
LOOKBACK_DAYS = 60
LOOKFORWARD_DAYS = 30

# Get stock list
stocks = get_nasdaq100_constituents(
    ADJ_DATE, LOOKBACK_DAYS, LOOKFORWARD_DAYS,
)

# Get portfolio model
model = PortMomentum(
    stocks=stocks,
    adj_date=ADJ_DATE,
    stock_data_dir="stock_prices",
    lookback_days=LOOKBACK_DAYS,
    window_days=30,
    window_overlap_days=15,
    weight_upper_limit=0.08,
    r_base=0.05 / 365,
    alpha=5.0, # penalty multiplier term
    beta=1.0, # penalty multiplier term
    xi=1.0, # multiplier as in the formula
)

Chose 101 of 102 stocks


In [3]:
# Solve on Dirac-3
solver = Dirac3ContinuousCloudSolver()
response = solver.solve(
    model,
    sum_constraint=100,
    relaxation_schedule=2,
    solution_precision=None,
    num_samples=10,
)
sol = response["results"]["solutions"][0][:len(stocks)]

print(response)

weight_hash = {}
for i in range(len(stocks)):
    weight_hash[stocks[i]] = sol[i] / 100.0

tot_weight = sum(weight_hash.values())

if tot_weight != 1.0:
    for stock in stocks:
        weight_hash[stock] = weight_hash[stock] / tot_weight

weight_df = pd.DataFrame(
    {
        "Stock": [item for item in weight_hash.keys()],
        "Allocation": [
            weight_hash[item] for item in weight_hash.keys()
        ],
    }
)
weight_df["Date"] = ADJ_DATE
weight_df = weight_df[weight_df["Allocation"] > 0]

ret_df = get_port_stats(weight_df, 30)

print(ret_df)


2025-03-15 10:16:57 - Dirac allocation balance = 0 s (unmetered)
2025-03-15 10:16:57 - Job submitted: job_id='67d5a7f900e804f113aefcc1'
2025-03-15 10:16:57 - QUEUED
2025-03-15 10:17:00 - RUNNING
2025-03-15 10:20:12 - COMPLETED
2025-03-15 10:20:15 - Dirac allocation balance = 0 s (unmetered)
{'job_info': {'job_id': '67d5a7f900e804f113aefcc1', 'job_submission': {'problem_config': {'normalized_qudit_hamiltonian_optimization': {'polynomial_file_id': '67d5a7f7236cc2225ac90ce5'}}, 'device_config': {'dirac-3_normalized_qudit': {'num_samples': 10, 'relaxation_schedule': 2, 'sum_constraint': 100}}}, 'job_status': {'submitted_at_rfc3339nano': '2025-03-15T16:16:57.428Z', 'queued_at_rfc3339nano': '2025-03-15T16:16:57.429Z', 'running_at_rfc3339nano': '2025-03-15T16:16:57.676Z', 'completed_at_rfc3339nano': '2025-03-15T16:20:11.598Z'}, 'job_result': {'file_id': '67d5a8bb236cc2225ac90ce9', 'device_usage_s': 84}}, 'status': 'COMPLETED', 'results': {'counts': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'energies': 